In [5]:
from stitch2d import StructuredMosaic
import pandas as pd
import numpy as np
import scanpy as sc
import anndata

# Image stitching

Stitch the images

Modify the **location** of your files

Modify the **dim** according to how many images you want in one row

In [7]:
mosaic = StructuredMosaic("C:/Users/Wally/Documents/Research/Research-Lung/Lung5_Rep1/Lung5_Rep1-Flat_files_and_images/CellComposite/",dim=5,origin="lower left",direction="horizontal",pattern='raster')

In [8]:
#visualize
mosaic

In [ ]:
#Save the stitched image

In [9]:
mosaic.save("mosaic5_1.png")

# Stitching the coordinates

Declaring the function for the coordinate stitching

The input is the dataframe with the coordinates. It should have the columns CenterX_global_px, CenterY_global_px, CenterX_local_px, CenterY_local_px

The input should also include the number of images per row -1, since python starts counting from 0. For example, I want 5 images in one row, my input should be 5-1=4

In [10]:
#For modifying the coordinates to plot in the stitched image input the dataframe with the coordinates 
#and the number of images per row -1

def stitching(df,imrow):
    df['cellid']=df.index
    df['cellid']=df['cellid'].astype(str)
    df[['fov','cellid']]=df.cellid.str.split('-',n=1,expand=True)
    v1=0
    v2=0
    for j in df['fov'].unique():
        print('Calculating for image: ',j)
        x=5471
        y=3647
        a=df[df['fov']==j]
        #print(a)
        if j==1:
            df.loc[df['fov']==j,'CenterX_global_px']=df.loc[df['fov']==j,'CenterX_local_px']
            df.loc[df['fov']==j,'CenterY_global_px']=df.loc[df['fov']==j,'CenterY_local_px']
            v1=v1+1
        elif v1>0 & v2>=0:
            df.loc[df['fov']==j,'CenterX_global_px']=df.loc[df['fov']==j,'CenterX_local_px']+(v1*x)
            df.loc[df['fov']==j,'CenterY_global_px']=df.loc[df['fov']==j,'CenterY_local_px']+(v2*y)
            if v1==imrow:
                v1=0
                v2=v2+1
            else:
                v1=v1+1
        elif v1==0 & v2>1:
            df.loc[df['fov']==j,'CenterX_global_px']=df.loc[df['fov']==j,'CenterX_local_px']
            df.loc[df['fov']==j,'CenterY_global_px']=df.loc[df['fov']==j,'CenterY_local_px']+(v2*y)
        elif v1>=0 & v2>=0:
            df.loc[df['fov']==j,'CenterX_global_px']=df.loc[df['fov']==j,'CenterX_local_px']+(v1*x)
            df.loc[df['fov']==j,'CenterY_global_px']=df.loc[df['fov']==j,'CenterY_local_px']+(v2*y)
            if v1==imrow:
                v1=0
                v2=v2+1
            else:
                v1=v1+1
    print('Done!')
    return(df)

Load the data that we generated using R

In [31]:
adata=sc.read_h5ad('Lung5-1-RNA-new-sct.h5ad')

In [32]:
#Visualize the data
adata.obs

,orig.ident,nCount_RNA,nFeature_RNA,fov,cell_ID,Area,AspectRatio,CenterX_local_px,CenterY_local_px,CenterX_global_px,...,Max.CD45,Mean.CD3,Max.CD3,Mean.DAPI,Max.DAPI,percent.mt,nCount_SCT,nFeature_SCT,SCT_snn_res.1.5,seurat_clusters
1-7,0,419.0,153,1,7,8525,1.38,714,3604,3902.888889,...,742,10,662,4781,22256,0.238663,418.0,153,5,5
1-53,0,359.0,146,1,53,4727,1.14,2788,3570,5976.888889,...,1761,4,537,13918,43001,0.557103,384.0,146,26,26
1-59,0,281.0,151,1,59,4907,0.82,4963,3561,8151.888889,...,1683,0,228,2507,16435,2.135231,358.0,151,21,21
1-64,0,254.0,114,1,64,2950,1.25,1439,3557,4627.888889,...,1042,6,648,10377,26876,0.393701,367.0,114,6,6
1-67,0,282.0,97,1,67,3854,0.86,1213,3556,4401.888889,...,1023,22,782,11727,35702,0.000000,366.0,97,11,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30-4709,0,304.0,132,30,4709,4163,1.13,5155,101,30232.777778,...,1419,0,34,7631,20532,0.657895,369.0,132,4,4
30-4714,0,397.0,131,30,4714,3058,1.18,2356,76,27433.777778,...,540,0,109,4213,18672,1.007557,401.0,131,6,6
30-4716,0,482.0,165,30,4716,5310,0.86,629,68,25706.777778,...,9533,0,109,7897,25932,0.414938,443.0,165,22,22
30-4725,0,415.0,165,30,4725,3656,1.08,1510,57,26587.777778,...,902,0,28,6695,23049,0.481928,415.0,165,1,1


In [33]:
adata.obs=stitching(adata.obs,4)

Calculating for image:  1
Calculating for image:  2
Calculating for image:  3
Calculating for image:  4
Calculating for image:  5
Calculating for image:  6
Calculating for image:  7
Calculating for image:  8
Calculating for image:  9
Calculating for image:  10
Calculating for image:  11
Calculating for image:  12
Calculating for image:  13
Calculating for image:  14
Calculating for image:  15
Calculating for image:  16
Calculating for image:  17
Calculating for image:  18
Calculating for image:  19
Calculating for image:  20
Calculating for image:  21
Calculating for image:  22
Calculating for image:  23
Calculating for image:  24
Calculating for image:  25
Calculating for image:  26
Calculating for image:  27
Calculating for image:  28
Calculating for image:  29
Calculating for image:  30
Done!


Make sure the coordinates were correctly stitched

In [34]:
adata.obs[['CenterX_local_px', 'CenterY_local_px', 'CenterX_global_px', 'CenterY_global_px']]

,CenterX_local_px,CenterY_local_px,CenterX_global_px,CenterY_global_px
1-7,714,3604,714.0,3604.0
1-53,2788,3570,2788.0,3570.0
1-59,4963,3561,4963.0,3561.0
1-64,1439,3557,1439.0,3557.0
1-67,1213,3556,1213.0,3556.0
...,...,...,...,...
30-4709,5155,101,27039.0,18336.0
30-4714,2356,76,24240.0,18311.0
30-4716,629,68,22513.0,18303.0
30-4725,1510,57,23394.0,18292.0


Make sure the _index in adata.var is actually the index

In [35]:
adata.var

,SCT_features,_index,features
0,AATK,AATK,AATK
1,ABL1,ABL1,ABL1
2,ABL2,ABL2,ABL2
3,ACE,ACE,ACE
4,ACE2,ACE2,ACE2
...,...,...,...
975,NegPrb19,NegPrb19,NegPrb19
976,NegPrb20,NegPrb20,NegPrb20
977,NegPrb21,NegPrb21,NegPrb21
978,NegPrb22,NegPrb22,NegPrb22


In [36]:
adata.var=adata.var.set_index('_index')
adata.var

,SCT_features,features
_index,,
AATK,AATK,AATK
ABL1,ABL1,ABL1
ABL2,ABL2,ABL2
ACE,ACE,ACE
ACE2,ACE2,ACE2
...,...,...
NegPrb19,NegPrb19,NegPrb19
NegPrb20,NegPrb20,NegPrb20
NegPrb21,NegPrb21,NegPrb21


Save the data with the stitched coordinates.

In [37]:
adata.__dict__['_raw'].__dict__['_var'] = adata.__dict__['_raw'].__dict__['_var'].rename(columns={'_index': 'features'})

In [38]:
adata.write('Lung-data-stitched.h5ad')